In [32]:
#1
class Foo:
  def __getitem__(self, pos):
    return range(0,30,10)[pos]


f = Foo()
f[1]
for i in f: print(i)

20 in f


0
10
20


True

In [33]:
15 in f

False

In [34]:
#2
import collections

Card = collections.namedtuple("Card",['rank','suit'])

class FrenchDeck:
  ranks = [str(n) for n in range(2,11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()

  def __init__(self):
    self._cards = [Card(rank,suit) for suit in self.suits for rank in self.ranks]

  def __len__(self):
    return len(self._cards)

  def __getitem__(self, position):
    return self._cards[position]

In [35]:
#3
from random import shuffle
#from frenchdeck import FrenchDeck

deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [36]:
#4
def set_card(deck, position, card):
  deck._cards[position] = card
FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='J', suit='clubs'),
 Card(rank='2', suit='hearts'),
 Card(rank='8', suit='hearts'),
 Card(rank='3', suit='spades'),
 Card(rank='8', suit='clubs')]

In [37]:
#5
import collections

Card = collections.namedtuple("Card",['rank','suit'])

class FrenchDeck2():
  ranks = [str(n) for n in range(2,11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()

  def __init__(self):
    self._cards = [Card(rank,suit) for suit in self.suits for rank in self.ranks]

  def __len__(self):
    return len(self._cards)

  def __getitem__(self, position):
    return self._cards[position]

  def __setitem__(self, position, card):
    self._cards[position] = card

  def __delitem__(self, position):
    del self._cards[position]

  def insert(self, position, value):
    self._cards.insert(position, value)

In [38]:
#6
import abc

class Tombola(abc.ABC):

  @abc.abstractmethod
  def load(self,iterable):
    """Add items from an iterable"""

  @abc.abstractmethod
  def pick(self):
    """Remove item at random, returning it.
    This method should raise 'LookupError' when the instance is empty
    """
  def loaded(self):
    """Return 'True' if there's at least 1 item, 'False' otherwise"""
    return bool(self.inspect())

  def inspect(self):
    """Return a sorted tuple with the items currently inside"""
    items = []
    while True:
      try:
        items.append(self.pick())
      except LookupError:
        break
    self.load(items)
    return tuple(sorted(items))

In [40]:
#7
#from tombola import Tombola

class BingoCage(Tombola):
  def __init__(self, items):
    self._randomizer = random.SystemRandom()
    self._items = []
    self.load(items)
  def load(self, items):
    self._items.extend(items)
    self._randomizer.shuffle(self._items)
  def pick(self):
    try:
      return self._items.pop
    except IndexError:
      raise LookupError('pick from empty BingoCage')

    def __call__(self):
      self.pick()

In [43]:
#8
import random

#from tombola import Tombola

class LotteryBlower(Tombola):
  def __init__(self, iterable):
    self._balls = list(iterable)

  def load(self,iterable):
    self._balls.extend(iterable)

  def pick(self):
    try:
      position = random.randrange(len(self._balls))

    except ValueError:
      raise LookupError('pick from empty BingoCage')
    return self._balls.pop(position)

  def loaded(self):
    return self._balls.pop(self._balls)

  def inspect(self):
    return tuple(sorted(self._balls))

In [45]:
#8
from random import randrange

#from tombola import Tombola

@Tombola.register
class TomboList(list):

  def pick(self):
    if self:
      position = randrange(len(self))
      return self.pop(position)
    else:
      raise LookupError('pop from empty TomboList')

  load = list.extend

  def loaded(self):
      return bool(self)

  def inspect(self):
      return tuple(sorted(self))